# Prepare NSF-OCE data for processing
Created by Ivan Lima on 2020-03-16 14:47:19 

In this notebook we:

- Clean and make PI and co-PI names more consistent in format.
- Finish cleaning Abstracts (remove html tags)
- Group awards by abstract:
    - Compute total amount awarded by adding amount awarded for each individual award.
    - Main PI, organization and program are taken from the award with the highest amount awarded.
    - Compute the number of awards in each group.
    - Collect PI (besides main PI) and co-PI names for grouped awards.
- Clean Programs and make them more consistent in format.
- Extract main Program and compute number of programs for each award
- Adjust total amount awarded for inflation to 2019 U.S. dollars.

In [1]:
import pandas as pd
import numpy as np
import datetime, re
from tqdm import tnrange, notebook
pd.options.display.max_columns = 50
print('Last updated on {}'.format(datetime.datetime.now().ctime()))

Last updated on Sat Mar 28 18:35:06 2020


## Read NSF-OCE data

In [2]:
data_types = {'Title':'string', 'Programs':'string', 'PI':'string', 'Co-PIs':'string',
              'Organization':'string', 'State':'string', 'Instrument':'string', 'AwardedAmount':'string',
              'ARRA_Amount':'string', 'Abstract':'string'}
awards = pd.read_csv('data/awards_1985-2020_clean_edited.csv', index_col=0, parse_dates=[3,4],
                     dtype=data_types)

# convert strings to floats
awards['AwardedAmount'] = [np.float(x) for x in awards.AwardedAmount.str.lstrip('$').str.replace(',','')]
awards['ARRA_Amount'] = [np.float(x) for x in awards.ARRA_Amount.str.lstrip('$').str.replace(',','')]

# remove awards with missing abstracts
awards = awards[awards.Abstract.notnull()]

# sort awards by abstract and amount awarded
awards = awards.sort_values(['Abstract', 'AwardedAmount'])

awards.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14717 entries, 8911427 to 844394
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Title          14717 non-null  string        
 1   Programs       14691 non-null  string        
 2   StartDate      14717 non-null  datetime64[ns]
 3   EndDate        14717 non-null  datetime64[ns]
 4   PI             14685 non-null  string        
 5   Co-PIs         4968 non-null   string        
 6   Organization   14717 non-null  string        
 7   State          14541 non-null  string        
 8   Instrument     14717 non-null  string        
 9   AwardedAmount  14717 non-null  float64       
 10  ARRA_Amount    14717 non-null  float64       
 11  Abstract       14717 non-null  string        
dtypes: datetime64[ns](2), float64(2), string(8)
memory usage: 1.5 MB


## Clean and make PI's and co-PIs names more consistent

In [3]:
# remove "Jr" from names
p = r'(,)*\s*Jr(\.)*\s*'
awards.loc[awards.PI.str.contains(r'Jr').fillna(0),'PI'] = [
    re.sub(p,'',s)for s in awards.loc[awards.PI.str.contains(r'Jr').fillna(0),'PI']]
awards.loc[awards['Co-PIs'].str.contains(r'Jr').fillna(0),'Co-PIs'] = [
    re.sub(p,'',s)for s in awards.loc[awards['Co-PIs'].str.contains(r'Jr').fillna(0),'Co-PIs']]

# remove "." from initials
p = r'\.'
awards.loc[awards.PI.str.contains(p).fillna(0),'PI'] = [
    ' '.join(s.replace('.',' ').split()) for s in awards.loc[awards.PI.str.contains(p).fillna(0),'PI']]
awards.loc[awards['Co-PIs'].str.contains(p).fillna(0),'Co-PIs'] = [
    ','.join([' '.join(re.sub(p,' ',s0).split())
              for s0 in s.split(',')]) for s in awards.loc[awards['Co-PIs'].str.contains(p).fillna(0),'Co-PIs']]

# remove space from dashes ("-")
awards.loc[awards.PI.str.contains(r'-').fillna(0),'PI'] = [
    re.sub(r'\s*-\s*','-',s) for s in awards.loc[awards.PI.str.contains(r'-').fillna(0),'PI']]

# remove parenthesis
# awards.loc[awards.PI.str.contains(r'\(').fillna(0),'PI'] = [
#     re.sub(r'[()]+','',s) for s in awards.loc[awards.PI.str.contains(r'\(').fillna(0),'PI']]

# names start with upper case
awards.loc[awards.PI.str.contains('JOAQUIM GOES').fillna(0),'PI'] = [
    s.title() for s in awards.loc[awards.PI.str.contains('JOAQUIM GOES').fillna(0),'PI']]

awards.loc[awards.PI.str.contains('penelope dalton').fillna(0),'PI'] = [
    s.title() for s in awards.loc[awards.PI.str.contains('penelope dalton').fillna(0),'PI']]

for p in [r'ANDREW LEISING', r'LISA BALLANCE', r'FEILI LI']:
    awards.loc[awards['Co-PIs'].str.contains(p).fillna(0),'Co-PIs'] = [
        ','.join([s0.title() for s0 in s.split(',')]) 
        for s in awards.loc[awards['Co-PIs'].str.contains(p).fillna(0),'Co-PIs']]

# remove extra empty spaces
awards.loc[awards.PI.notnull(),'PI'] = [' '.join(s.split())
                                        for s in awards.loc[awards.PI.notnull(),'PI']]

## Finish cleaning Abstracts

In [4]:
# remove html tags
awards.loc[awards.Abstract.str.contains('<br/>').fillna(0),'Abstract'] = [
    s.replace('<br/>',' ') for s in awards.loc[awards.Abstract.str.contains('<br/>').fillna(0),'Abstract']]

# remove multiple empty spaces
awards.loc[awards.Abstract.notnull(),'Abstract'] = [
    ' '.join(s.split()) for s in awards.loc[awards.Abstract.notnull(),'Abstract']]

In [5]:
# len(awards.loc[awards.Abstract.str.contains('OCE\s*-\s*\d{6,}'),'Abstract'])
# for i, s in awards.loc[awards.Abstract.str.contains('OCE\s*-\s*\d{6,}'),'Abstract'].iteritems():
#     print('{}\n{}\n'.format(i,s))

## Group awards by abstract

**Note:** Collaborative Research awards are those in which investigators from two or more organizations collaborate on a single research project. These awards share the same abstract but the different organizations receive separate awards. Here we group awards by abstract, so Collaborative Research awards that are part of the same project are grouped into one award and the total amount awarded is the sum of the amounts awarded to each organization. The main PI and Organization for each group/project is assumed to be the one with the highest amount awarded.

In [6]:
def top_award(df, col='AwardedAmount'):
    cols = ['AwardNumber','Title','Programs','StartDate','EndDate',
            'PI','Organization','State','Instrument']
    return df.sort_values(col)[-1:][cols]

# def concat_names(df, col='PI'):
#     names = ','.join(df[col].dropna().to_list())
#     if len(names) == 0:
#         return np.nan
#     else:
#         return sorted(set([s.strip() for s in names.split(',')]))

# function to collect PIs (besides the main PI) and co-PIs for grouped awards
def concat_names2(df):
    name_list = df['PI'].dropna().to_list() + df['Co-PIs'].dropna().to_list()
    names = sorted(set([x.strip() for x in sum([s.split(',') for s in name_list], [])])) # remove spaces
    main_pi = df.sort_values('AwardedAmount')[-1:]['PI'].item()
    if pd.notna(main_pi):
        if main_pi in names:
            names.remove(main_pi)
    if len(names) == 0:
        return np.nan
    else:
        return ','.join(names)

awards_grouped = awards.reset_index().groupby('Abstract').apply(top_award)
awards_grouped = awards_grouped.reset_index(level=1, drop=True)

# compute number of awards in each group
awards_grouped['n_awards'] = awards.groupby('Abstract').size()

# collect remaining PI and co-PI names in each group
awards_grouped['co-PIs'] = awards.reset_index().groupby('Abstract').apply(concat_names2)

# compute number of co-PIs
awards_grouped.loc[awards_grouped['co-PIs'].notnull(),'num_co-PIs'] = (
    awards_grouped.loc[awards_grouped['co-PIs'].notnull(),'co-PIs'].str.split(',').map(len))

# compute total amount awarded for each group
awards_grouped['total_amount'] = awards.reset_index().groupby('Abstract')['AwardedAmount'].sum()

# compute total ARRA amount awarded for each group
awards_grouped['total_ARRA'] = awards.reset_index().groupby('Abstract')['ARRA_Amount'].sum()

# use AwardNumber as index
awards_grouped = awards_grouped.reset_index().set_index('AwardNumber')

print('Total number of awards after grouping: {}\n'.format(len(awards_grouped)))

awards_grouped.info()

Total number of awards after grouping: 11513

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11513 entries, 8911427 to 844394
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Abstract      11513 non-null  object        
 1   Title         11513 non-null  string        
 2   Programs      11491 non-null  string        
 3   StartDate     11513 non-null  datetime64[ns]
 4   EndDate       11513 non-null  datetime64[ns]
 5   PI            11481 non-null  string        
 6   Organization  11513 non-null  string        
 7   State         11364 non-null  string        
 8   Instrument    11513 non-null  string        
 9   n_awards      11513 non-null  int64         
 10  co-PIs        5784 non-null   object        
 11  num_co-PIs    5784 non-null   float64       
 12  total_amount  11513 non-null  float64       
 13  total_ARRA    11513 non-null  float64       
dtypes: datetime64[ns](2), float64(3),

## Clean Programs and get main program and total number of programs for each award

In [7]:
# remove empty spaces separated by commas & convert everything to upper case
awards_grouped.loc[awards_grouped.Programs.notnull(),'Programs'] = [
     (',').join([x.strip().upper() for x in s.split(',') if x != ' '])
     for s in awards_grouped.loc[awards_grouped.Programs.notnull(),'Programs']]

# set award program to first in the list
awards_grouped.loc[awards_grouped.Programs.notnull(),'program'] = [
    s.split(',')[0] for s in
    awards_grouped.loc[awards_grouped.Programs.notnull(),'Programs']]

# get number of programs for each award
awards_grouped.loc[awards_grouped.Programs.notnull(),'n_programs'] = [
    len(s.split(',')) for s in awards_grouped.loc[awards_grouped.Programs.notnull(),'Programs']]

## Adjust amount awarded for inflation

In [8]:
cpi = pd.read_excel('data/CPI_1985-2019.xlsx', skiprows=range(11), index_col=0) # consumer price index data
rel_cpi = cpi.Annual/cpi.loc[2019,'Annual']                                     # use 2019 as reference year
rel_cpi_map = {yr:val for yr, val in zip(rel_cpi.index, rel_cpi.values)}
awards_grouped['tot_amount_adjusted'] = (awards_grouped.total_amount /
                                   awards_grouped.StartDate.map(lambda x: rel_cpi_map[x.year]))
awards_grouped['tot_ARRA_adjusted'] = (awards_grouped.total_ARRA /
                                 awards_grouped.StartDate.map(lambda x: rel_cpi_map[x.year]))

## Save data set to CSV file

In [9]:
awards_grouped.to_csv('data/awards_1985-2020_clean_edited_grouped.csv', encoding='utf-8-sig')
awards_grouped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11513 entries, 8911427 to 844394
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Abstract             11513 non-null  object        
 1   Title                11513 non-null  string        
 2   Programs             11491 non-null  string        
 3   StartDate            11513 non-null  datetime64[ns]
 4   EndDate              11513 non-null  datetime64[ns]
 5   PI                   11481 non-null  string        
 6   Organization         11513 non-null  string        
 7   State                11364 non-null  string        
 8   Instrument           11513 non-null  string        
 9   n_awards             11513 non-null  int64         
 10  co-PIs               5784 non-null   object        
 11  num_co-PIs           5784 non-null   float64       
 12  total_amount         11513 non-null  float64       
 13  total_ARRA           115